<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/GraphWriter_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Change accordingly
import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Install all requirements.

In [2]:
#!pip install -r requirements.txt
!pip install torchtext -U
!pip install spacy==2.1.0
!pip install neuralcoref allennlp hnswlib allennlp-models
!pip install wandb
!python -m spacy download en

#  RESTART RUNTIME AFTER INSTALLATION

     |████████████████████████████████| 296kB 8.6MB/s 
     |████████████████████████████████| 481kB 16.6MB/s 
     |████████████████████████████████| 286kB 28.6MB/s 
     |████████████████████████████████| 266kB 31.0MB/s 
     |████████████████████████████████| 675kB 30.7MB/s 
     |████████████████████████████████| 317kB 51.3MB/s 
     |████████████████████████████████| 296kB 57.4MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
     |████████████████████████████████| 3.8MB 51.7MB/s 
  ERROR: Failed building wheel for hnswlib
  Running setup.py clean for hnswlib
  Created wheel for jsonnet: filename=jsonnet-0.16.0-cp36-cp36m-linux_x86_64.whl size=3321603 sha256=4dabf8bf47283c8493b5ee39073cdf865fb0bb128874eed424d932a84f6c4933
  Stored in directory: /root/.cache/pip/wheels/64/a9/43/bc5e0463deeec89dfca928a2a64595f1bdb520c891f6fbd09c
  Created wheel for overrides: filename=overrides-3.0.0-cp36-none-any.whl size=5669 sha256

Import everything

In [2]:
import neuralcoref
import spacy
import re
import pickle
import pandas as pd
import gc
from Answer import Answer
import utils

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
INFO:absl:Using /tmp/tfhub_modules to cache modules.


Setup everything for coref

In [ ]:
'''
nlp = spacy.load('en')
neuralcoref.add_to_pipe(nlp)

identifier='coref'
last_check=0
step_size=1000

def load_pickles(identifier, checkpoint):
  if os.path.exists("df"+identifier+str(checkpoint)+".pickle"):
    df = pd.read_pickle("df"+identifier+str(checkpoint)+".pickle")
  return df

def dump_pickles(identifier, checkpoint, df):
  df.to_pickle("df"+identifier+str(checkpoint)+".pickle")
  gc.collect()
  return 

def make_corefs(content):
    content=str(re.sub("\.(?=\s[a-z0-9]|\sI[\W\s])", ",", content))
    doc=nlp(content)
    return doc._.coref_resolved

df = pd.read_pickle("final_data.pkl")
'''


Do coref

In [6]:
'''
for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  for j in range(step_size):
    df.iloc[step_size*(i-1)+j]['answer']=make_corefs(df.iloc[step_size*(i-1)+j]['answer'])
  dump_pickles(identifier,i,df)
print('done with ordered')
for j in range(df.shape[0]-int(df.shape[0]/step_size)*step_size):
  df.iloc[step_size*int(df.shape[0]/step_size)+j]['answer']=make_corefs(df.iloc[step_size*int(df.shape[0]/step_size)+j]['answer'])
df.to_pickle(identifier+".pkl")
'''
df = pd.read_pickle("coref.pkl")
df.head()

,question,answer,party
0,Should abortion be illegal?,Mr. Speaker. Congress now has an opportunity t...,R
1,Should abortion be illegal?,"Mr. Speaker, I rise today to express my opposi...",R
2,Should abortion be illegal?,"so eloquently pointed out earlier, partialbirt...",R
3,Should abortion be illegal?,"Mr. Chairman, I rise in reluctant support of t...",D
4,Should abortion be illegal?,Will the Senator yield to answer a question so...,D


Setup for parsing

In [4]:
identifier='oie'
last_check=1
step_size=100
#df is still from the cell above above

def parse_entry(entry,verb_dict,verb_list):
    #print(entry)
    #print("+"*20)
    result=dict()
    result['question']=' '.join([token.text for token in utils.sp(entry['question'])])
    phrase_corpus, triplet_id, parsed_text,parsed=Answer(entry['answer']).create_test(verb_dict,verb_list)
    result['corpus']=' ; '.join(phrase_corpus)
    result['tags']=' '.join(['<phrase>']*len(phrase_corpus))
    result['triplet_id']=' ; '.join([re.sub('\,','',str(x))[1:-1] for x in triplet_id])
    result['parsed_text']=parsed_text
    result['parsed']=' '.join([str(x) for x in parsed])
    return result

def load_pickles(identifier, checkpoint):
  if os.path.exists("verb_dict"+identifier+str(checkpoint)+".pickle"):
    with open('verb_dict'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_dict = pickle.load(handle)
    with open('verb_list'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_list = pickle.load(handle)
    with open('result'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      result = pickle.load(handle)
  else:
    print('File does not exist yet.')
    verb_dict=dict()
    verb_list=[]
    result=[]
  return verb_dict, verb_list, result

def dump_pickles(identifier, checkpoint, verb_dict, verb_list, result):
  with open("verb_dict"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("verb_list"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("result"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return 

verb_dict = pickle.load(open("verb_dict.pickle", "rb"))
verb_list = None

Process data

In [6]:
result = []
for i in range(last_check+1,int(df.shape[0]/step_size)):
  print(i)
  df[step_size*(i-1):step_size*i].apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
  dump_pickles(identifier,i,verb_dict,verb_list,result)
print('done with ordered')
df = df[df['answer'].apply(lambda x: len(x)) < 1000]
df.apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
dump_pickles(identifier,1+df.shape[0]/step_size,verb_dict,verb_list,result)

done with ordered
DO
DO
DO
BELLYACHE
DO
DISFAVOR
DO
DO
GENDERBASE
DO
GENDERBASE
DISCUSS
DENOTE
DO
DO
DO
DO
DO
COOK
COOK
DO
INFECT
DO


Post-processing of data

In [7]:
df=pd.DataFrame(result) 
df=df.sample(frac=1, random_state=36)
df=df.dropna(axis=0, how='any')

Write to variables

In [8]:
identifier='../GraphWriter-master/data/preprocessed'
df.to_csv("/content/drive/My Drive/GraphWriter-master/data/preprocessed.test.tsv", sep='\t', index=False, header=False)

Train with GraphWriter

In [ ]:
#!python ../GraphWriter-master/train.py -save ../GraphWriter-master/partisanResponses

Use file with lowest vloss in the folder "partisanResponses" of GraphWriter-master to generate responses on test data.


E.g. in the next cell "9.vloss-3.980539.lr-0.1" was used.

In [12]:
os.chdir('/content/drive/My Drive/GraphWriter-master')
!python generator.py -save=partisan-responses/first_20_ckpt/9.vloss-3.980539.lr-0.1

2020-07-24 20:13:56.955420: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Loading Data from  data/preprocessed.train.tsv
data/
preprocessed.train.tsv
building vocab
done
Vocab sizes:
src 6847
ent 18458
nerd 4
rel 18429
out 9801
graph
0 42
/content/drive/My Drive/GraphWriter-master/models/newmodel.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cx = torch.tensor(hx)
That is correct . What we are trying to do is restate in simple , easy to understand language THE BALANCE that THE COURTS , I think , have properly struck between <unk> THE PURPOSE OF THE 21ST AMENDMENT and at the same time making sure that WE do not subtract in any way FROM THE INTERSTATE COMME

In [13]:
df.head()

,question,corpus,tags,triplet_id,parsed_text,parsed
28,What do you think about the current president ?,the courts ; the balance ; the purpose of the ...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 252 1 ; 0 500 2 ; 3 7406 4 ; 3 1388 5 ; 3 12...,That is correct . What we are trying to do is ...,6 -1 0 1 7 2 8 3 4 9 5 10 11 6 -1
24,Should every American have equal opportunities...,by Madam Speaker and the House Republican lead...,<phrase> <phrase> <phrase> <phrase>,0 905 1 ; 1 2039 2 ; 1 8310 1 ; 1 58 3,"Madam Speaker , <phrase_1> have been assured <...",0 1 5 4 -1 2 6 7 4 -1 4 -1 4 -1 3 8 4 -1 4 -1 ...
11,What do you think about the current president ?,Mr. Chairman ; my time ; we all ; with ; citiz...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 3932 1 ; 2 285 3 ; 4 5158 5 ; 2 285 6,"<phrase_0> , reclaiming <phrase_1> for a momen...",0 1 8 7 -1 7 -1 7 -1 2 3 9 4 5 10 7 -1 6 11 7 -1
39,What is the biggest threat to America ?,I ; Senator LIEBERMAN ; a question ; by the Ir...,<phrase> <phrase> <phrase> <phrase> <phrase>,0 4201 1 ; 0 1309 2 ; 3 4790 4,"Mr. President , <phrase_0> see <phrase_1> . Se...",0 1 6 5 -1 5 -1 2 7 5 -1 3 4 8 5 -1 5 -1
7,Should we have higher taxes for higher incomes ?,"the Democrats ; the Tax Code ; share""-I ; that...",<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 82 1 ; 0 237 1 ; 2 15 3 ; 3 5158 1 ; 1 565 3...,"So when the Democrats , in 1993 , said . "" <ph...",7 -1 0 1 8 9 2 3 10 11 12 4 5 13 14 7 -1 6 15 ...


In [16]:
with open("/content/drive/My Drive/GraphWriter-master/outputs/9.vloss-3.980539.lr-0.1.inputs.beam_predictions.cmdline") as f:
  gw_gen_answers = f.readlines()

In [19]:
df['GW_ANS'] = gw_gen_answers

In [24]:
pickle.dump(df, open("GW_output_200724.pkl", "wb"))

In [13]:
gw_df = pickle.load(open("GW_output_200724.pkl", "rb"))
gw_df.head(5)

,question,corpus,tags,triplet_id,parsed_text,parsed,GW_ANS
28,What do you think about the current president ?,the courts ; the balance ; the purpose of the ...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 252 1 ; 0 500 2 ; 3 7406 4 ; 3 1388 5 ; 3 12...,That is correct . What we are trying to do is ...,6 -1 0 1 7 2 8 3 4 9 5 10 11 6 -1,"mr. chairman , if the gentleman will yield , w..."
24,Should every American have equal opportunities...,by Madam Speaker and the House Republican lead...,<phrase> <phrase> <phrase> <phrase>,0 905 1 ; 1 2039 2 ; 1 8310 1 ; 1 58 3,"Madam Speaker , <phrase_1> have been assured <...",0 1 5 4 -1 2 6 7 4 -1 4 -1 4 -1 3 8 4 -1 4 -1 ...,"mr. president , i that i have to say that i ha..."
11,What do you think about the current president ?,Mr. Chairman ; my time ; we all ; with ; citiz...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 3932 1 ; 2 285 3 ; 4 5158 5 ; 2 285 6,"<phrase_0> , reclaiming <phrase_1> for a momen...",0 1 8 7 -1 7 -1 7 -1 2 3 9 4 5 10 7 -1 6 11 7 -1,"mr. chairman , if the gentleman will yield fur..."
39,What is the biggest threat to America ?,I ; Senator LIEBERMAN ; a question ; by the Ir...,<phrase> <phrase> <phrase> <phrase> <phrase>,0 4201 1 ; 0 1309 2 ; 3 4790 4,"Mr. President , <phrase_0> see <phrase_1> . Se...",0 1 6 5 -1 5 -1 2 7 5 -1 3 4 8 5 -1 5 -1,"mr. president , i that i me say to the senator..."
7,Should we have higher taxes for higher incomes ?,"the Democrats ; the Tax Code ; share""-I ; that...",<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 82 1 ; 0 237 1 ; 2 15 3 ; 3 5158 1 ; 1 565 3...,"So when the Democrats , in 1993 , said . "" <ph...",7 -1 0 1 8 9 2 3 10 11 12 4 5 13 14 7 -1 6 15 ...,"if the senator will allow that , let me say to..."


Will the Senator yield to answer a question so when WE debate WE , WE have AN UNDERSTANDING ? Like I said , this is the first time the Senator and I have stood nose to nose on this . Does the Senator believe abortion should be illegal ? 

Mr. Chairman , if the gentleman will yield to the gentleman from Texas , WE , WE , WE , and WE that may see that WE in the United States , to see AN UNDERSTANDING in the United States . If we can , WE we have AN UNDERSTANDING in the United States , and we can not do that , and WE we can see how WE we can see how WE WE can come to see how WE we can get AN UNDERSTANDING to see how WE can come to see how WE WE can come to see how WE we can , and WE WE can come to work and to see AN UNDERSTANDING in AN UNDERSTANDING that may see WE in the future . If we can get AN UNDERSTANDING that will see WE in the future , WE we can see AN UNDERSTANDING that WE can come to work and to see how WE WE can come to work and to see how WE WE can come to see how WE WE can come to see how WE WE can come to see how WE WE can come to work and see how WE can come to work and to see


In [4]:
df = df[df['answer'].apply(lambda x: len(x)) < 1000]
df.head(10)

NameError: ignored

In [11]:
df.loc[0, 'answer']

'Mr. Speaker. Congress now has an opportunity to put an end to one of the most barbaric acts known to mankind. Amazingly, this barbaric act is perfectly legal in the United States of America. this barbaric act is called partialbirth abortion. Actually this barbaric act is a procedure in which a baby is partially delivered, the doctor actually reaches in and turns a baby for a breech birth, and then is killed in a procedure too horrible to describe. Congress has voted twice to make it illegal, but the previous President vetoed it both times. Today we have a new President who will do the right thing and make partialbirth abortion illegal, but first we have to send the previous President the bill. we in the House will have the chance to vote on the bill later this summer, and we will do the right thing. It will be up to the other body to act. No one knows what they will do. It is time to make partialbirth abortion illegal. It should have been illegal long ago.'